In [1]:
import pandas as pd
import geopandas as gpd
import sys
sys.path.append('../lib/')
import math


import workers
import sweden
import scipy.interpolate

In [2]:
# Initialise an object for storing the ground-truth data including zones
data_sweden = sweden.GroundTruthLoader()
# Load zones
data_sweden.load_zones()
# Load ground-truth survey data into ODM form
data_sweden.load_odm()

In [3]:
data_sweden.odm.head()

,ozone,dzone,v_ij_gt,D_ij_gt
0,0114A0010,0114C1170,182551.736842,9.0
1,0114A0010,0163C1140,810292.236111,15.0
2,0114A0010,0163C1170,182927.555556,10.0
3,0114A0010,0163C1350,94404.100629,9.0
4,0114A0010,0163C1380,23397.135431,20.0


In [4]:
data_sweden.odm.loc[:, 'ozone_deso'] = data_sweden.odm.loc[:, 'ozone'].apply(lambda x: x[:4])
data_sweden.odm.loc[:, 'dzone_deso'] = data_sweden.odm.loc[:, 'dzone'].apply(lambda x: x[:4])
flows_gt = data_sweden.odm.groupby(['ozone_deso', 'dzone_deso'])['v_ij_gt'].sum().reset_index()
flows_gt.head()

,ozone_deso,dzone_deso,v_ij_gt
0,0114,0114,5.144816e+07
1,0114,0115,2.166490e+05
2,0114,0117,5.804400e+05
3,0114,0123,5.046360e+05
4,0114,0125,2.617775e+04


In [5]:
distances = workers.zone_distances(data_sweden.zones)

Calculating distances between zones...


In [6]:
# This gives a matrix-style dataframe
df_d = distances.unstack(level=1)
df_d.head(3)

zone,0114A0010,0114C1010,0114C1020,0114C1030,0114C1040,0114C1050,0114C1060,0114C1070,0114C1080,0114C1090,...,2584C1020,2584C1030,2584C1040,2584C1050,2584C1060,2584C1070,2584C1090,2584C1100,2584C1110,2584C1130
zone,,,,,,,,,,,,,,,,,,,,,
0114A0010,0.000000,4.795251,3.548429,4.160562,2.703179,4.783886,1.849689,2.923878,2.159032,1.674308,...,933.767042,933.828072,935.067203,935.416006,935.630309,935.620128,936.859131,936.431609,936.568693,937.598973
0114C1010,4.795251,0.000000,1.343363,0.686608,2.451822,1.246766,3.968650,2.079711,2.668393,3.291240,...,937.070798,937.137238,938.379341,938.733022,938.942200,938.922048,940.174915,939.742567,939.882250,940.918898
0114C1020,3.548429,1.343363,0.000000,0.657756,1.693341,1.362823,3.073683,0.737077,1.389539,1.969922,...,935.881658,935.947128,937.188698,937.541507,937.751603,937.733240,938.983622,938.552138,938.691355,939.726864


In [7]:
df_d.loc['0114A0010']['0114A0010']

0.0

In [8]:
data_sweden.odm.loc[:, 'distance'] = 0

In [9]:
data_sweden.odm.head()

,ozone,dzone,v_ij_gt,D_ij_gt,ozone_deso,dzone_deso,distance
0,0114A0010,0114C1170,182551.736842,9.0,0114,0114,0
1,0114A0010,0163C1140,810292.236111,15.0,0114,0163,0
2,0114A0010,0163C1170,182927.555556,10.0,0114,0163,0
3,0114A0010,0163C1350,94404.100629,9.0,0114,0163,0
4,0114A0010,0163C1380,23397.135431,20.0,0114,0163,0


In [10]:
for i in data_sweden.odm.index:
    data_sweden.odm.loc[i, 'distance'] = df_d[data_sweden.odm.loc[i, 'ozone']][data_sweden.odm.loc[i, 'dzone']]

In [11]:
data_sweden.odm.head()

,ozone,dzone,v_ij_gt,D_ij_gt,ozone_deso,dzone_deso,distance
0,0114A0010,0114C1170,182551.736842,9.0,0114,0114,2.301862
1,0114A0010,0163C1140,810292.236111,15.0,0114,0163,11.326053
2,0114A0010,0163C1170,182927.555556,10.0,0114,0163,11.993063
3,0114A0010,0163C1350,94404.100629,9.0,0114,0163,5.807288
4,0114A0010,0163C1380,23397.135431,20.0,0114,0163,4.713912


In [12]:
data_sweden.odm.loc[:, 'd_ij_gt'] = 0
data_sweden.odm.loc[:, 'D_ij_data_gt'] = 0
data_sweden.odm.loc[:, 'D_ij_sim_gt'] = 0

In [13]:
data_sweden.odm.head()

,ozone,dzone,v_ij_gt,D_ij_gt,ozone_deso,dzone_deso,distance,d_ij_gt,D_ij_data_gt,D_ij_sim_gt
0,0114A0010,0114C1170,182551.736842,9.0,0114,0114,2.301862,0,0,0
1,0114A0010,0163C1140,810292.236111,15.0,0114,0163,11.326053,0,0,0
2,0114A0010,0163C1170,182927.555556,10.0,0114,0163,11.993063,0,0,0
3,0114A0010,0163C1350,94404.100629,9.0,0114,0163,5.807288,0,0,0
4,0114A0010,0163C1380,23397.135431,20.0,0114,0163,4.713912,0,0,0


In [14]:
for i in data_sweden.odm.index:
    data_sweden.odm.loc[i, 'd_ij_gt'] = data_sweden.odm.loc[i, 'v_ij_gt'] * data_sweden.odm.loc[i, 'distance']

In [15]:
df_data = pd.read_csv('../results/distance_ratio_data.csv')
df_simulation = pd.read_csv('../results/distance_ratio_simulation.csv')

sp_data_sweden = scipy.interpolate.interp1d(df_data.loc[df_data.country == 'sweden', ['distance']].values.reshape(-1),
                                            df_data.loc[df_data.country == 'sweden', ['ratio']].values.reshape(-1), bounds_error = False, fill_value = 1.5)

sp_data_simulation = scipy.interpolate.interp1d(df_simulation.loc[:, ['distance']].values.reshape(-1),
                                                df_simulation.loc[:, ['ratio']].values.reshape(-1), bounds_error = False, fill_value = 1.5)

In [16]:
for i in data_sweden.odm.index:
    data_sweden.odm.loc[i, 'D_ij_data_gt'] = data_sweden.odm.loc[i, 'v_ij_gt'] * sp_data_sweden(data_sweden.odm.loc[i, 'distance'])

In [17]:
for i in data_sweden.odm.index:
    data_sweden.odm.loc[i, 'D_ij_sim_gt'] = data_sweden.odm.loc[i, 'v_ij_gt'] * sp_data_simulation(data_sweden.odm.loc[i, 'distance'])

In [18]:
data_sweden.odm.head()

,ozone,dzone,v_ij_gt,D_ij_gt,ozone_deso,dzone_deso,distance,d_ij_gt,D_ij_data_gt,D_ij_sim_gt
0,0114A0010,0114C1170,182551.736842,9.0,0114,0114,2.301862,4.202089e+05,2.989144e+05,2.647308e+05
1,0114A0010,0163C1140,810292.236111,15.0,0114,0163,11.326053,9.177413e+06,1.155347e+06,1.014611e+06
2,0114A0010,0163C1170,182927.555556,10.0,0114,0163,11.993063,2.193862e+06,2.599842e+05,2.282689e+05
3,0114A0010,0163C1350,94404.100629,9.0,0114,0163,5.807288,5.482318e+05,1.431992e+05,1.240085e+05
4,0114A0010,0163C1380,23397.135431,20.0,0114,0163,4.713912,1.102920e+05,3.583662e+04,3.116087e+04


In [19]:
travel_demand_d = data_sweden.odm.groupby(['ozone_deso', 'dzone_deso'])['d_ij_gt'].sum().reset_index()
travel_demand_d.head()

,ozone_deso,dzone_deso,d_ij_gt
0,0114,0114,5.951122e+07
1,0114,0115,1.752398e+06
2,0114,0117,1.281179e+07
3,0114,0123,5.125630e+06
4,0114,0125,7.097814e+05


In [20]:
travel_demand_D_survey = data_sweden.odm.groupby(['ozone_deso', 'dzone_deso'])['D_ij_data_gt'].sum().reset_index()
travel_demand_D_survey.head()

,ozone_deso,dzone_deso,D_ij_data_gt
0,0114,0114,8.610559e+07
1,0114,0115,3.147188e+05
2,0114,0117,7.765934e+05
3,0114,0123,7.235744e+05
4,0114,0125,3.421247e+04


In [21]:
travel_demand_D_sim = data_sweden.odm.groupby(['ozone_deso', 'dzone_deso'])['D_ij_sim_gt'].sum().reset_index()
travel_demand_D_sim.head()

,ozone_deso,dzone_deso,D_ij_sim_gt
0,0114,0114,8.120739e+07
1,0114,0115,2.775874e+05
2,0114,0117,7.071377e+05
3,0114,0123,6.369846e+05
4,0114,0125,3.199180e+04


In [31]:
flows_gt.to_csv('../results/flows_gt.csv', index=None)
travel_demand_d.to_csv("../results/demand_d_GT.csv", index=None)
travel_demand_D_survey.to_csv("../results/demand_D_survey_GT.csv", index=None)
travel_demand_D_sim.to_csv("../results/demand_D_sim_GT.csv", index=None)

In [33]:
test1 = pd.DataFrame(pd.read_csv('../results/flows_gt.csv', converters={u'ozone_deso':str, u'dzone_deso':str, u'D_ij_sim_gt':float}))
test2 = pd.DataFrame(pd.read_csv('../results/demand_d_GT.csv', converters={u'ozone_deso':str, u'dzone_deso':str, u'D_ij_sim_gt':float}))
test3 = pd.DataFrame(pd.read_csv('../results/demand_D_survey_GT.csv', converters={u'ozone_deso':str, u'dzone_deso':str, u'D_ij_sim_gt':float}))
test4 = pd.DataFrame(pd.read_csv('../results/demand_D_sim_GT.csv', converters={u'ozone_deso':str, u'dzone_deso':str, u'D_ij_sim_gt':float}))

In [34]:
test1.head()

,ozone_deso,dzone_deso,v_ij_gt
0,0114,0114,5.144816e+07
1,0114,0115,2.166490e+05
2,0114,0117,5.804400e+05
3,0114,0123,5.046360e+05
4,0114,0125,2.617775e+04


In [35]:
test2.head()

,ozone_deso,dzone_deso,d_ij_gt
0,0114,0114,5.951122e+07
1,0114,0115,1.752398e+06
2,0114,0117,1.281179e+07
3,0114,0123,5.125630e+06
4,0114,0125,7.097814e+05


In [36]:
test3.head()

,ozone_deso,dzone_deso,D_ij_data_gt
0,0114,0114,8.610559e+07
1,0114,0115,3.147188e+05
2,0114,0117,7.765934e+05
3,0114,0123,7.235744e+05
4,0114,0125,3.421247e+04


In [37]:
test4.head()

,ozone_deso,dzone_deso,D_ij_sim_gt
0,0114,0114,8.120739e+07
1,0114,0115,2.775874e+05
2,0114,0117,7.071377e+05
3,0114,0123,6.369846e+05
4,0114,0125,3.199180e+04
